In [14]:
// #include "include/lamppp/lamppp.hpp"
#pragma cling add_include_path("include/lamppp/lamppp.hpp")
#include <vector>

In [15]:
  auto a =
      lmp::tensor::Tensor(std::vector<int>{1, 2, 3}, std::vector<size_t>{3, 1});
  // auto b = lmp::tensor::Tensor(std::vector<int>{4, 5}, std::vector<size_t>{2});

  // auto c = a + b;
  // std::cout << c << std::endl;

input_line_22:3:7: error: use of undeclared identifier 'lmp'
      lmp::tensor::Tensor(std::vector<int>{1, 2, 3}, std::vector<size_t>{3, 1});
      ^
In file included from input_line_5:1:
In file included from /home/nlin/miniconda3/envs/xeus-cling/include/xeus/xinterpreter.hpp:13:
In file included from /home/nlin/miniconda3/envs/xeus-cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/10.4.0/functional:62:
In file included from /home/nlin/miniconda3/envs/xeus-cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/10.4.0/vector:67:
/home/nlin/miniconda3/envs/xeus-cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/10.4.0/bits/stl_vector.h:401:66: error: no member named 'value' in 'std::is_same<int, int>'
      static_assert(is_same<typename remove_cv<_Tp>::type, _Tp>::value,
                    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
input_line_22:3:27: note: in instantiation of template class 'std::vector<int, std::allocator<int> >' requested here
    

Interpreter Error: 